# Analyzing Carbon Footprints in SQL

_Greenhouse gas emissions attributable to products&mdash;from food to sneakers to appliances&mdash;make up more than 75% of global emissions._

Cite: [The Carbon Catalogue](https://www.nature.com/articles/s41597-022-01178-9)

The dataset, which is publicly availably on [nature.com](https://www.nature.com/articles/s41597-022-01178-9), stores product carbon footprints (PCFs) for various companies. PCFs are the greenhouse gas emissions attributable to a given product, measured in CO<sub>2</sub> (carbon dioxide equivalent).

## Our Database

The database contains one table, which looks at PCFs by product as well as the stage of production these emissions occured in.

### `product_emissions`

| field                              | data_type |
|------------------------------------|-----------|
| id                                 | VARCHAR   |
| year                               | INT       |
| product_name                       | VARCHAR   |
| company                            | VARCHAR   |
| country                            | VARCHAR   |
| industry_group                     | VARCHAR   |
| weight_kg                          | NUMERIC   |
| carbon_footprint_pcf               | NUMERIC   |
| upstream_percent_total_pcf         | VARCHAR   |
| operations_percent_total_pcf       | VARCHAR   |
| downstream_percent_total_pcf       | VARCHAR   |

## 1: Coca-Cola's emissions

First, let's look at a small subset of the data: emissions reported by Coca-Cola. Coke is actually made up of multiple companies around the globe, so we'll make sure our query returns data for any company name that starts with "Coca-Cola". Coke used to report for every single different product it has, so we will limit the results to six.

- Select all fields from the `product_emissions` table where the company name begins with "Coca-Cola", limiting to the first six results.

In [25]:
-- Select all fields where the company name is Coca-Cola, limiting to the first six results
SELECT *
FROM product_emissions	
WHERE company LIKE 'Coca-Cola%'
LIMIT 6;

,id,year,product_name,company,country,industry_group,weight_kg,carbon_footprint_pcf,upstream_percent_total_pcf,operations_percent_total_pcf,downstream_percent_total_pcf
0,22710-1-2014,2014,Coca-Cola (all packaging and sizes),Coca-Cola HBC AG,Switzerland,"Food, Beverage & Tobacco",1.093,0.1673,84.28%,11.12%,4.60%
1,22710-1-2015,2015,Coca-Cola (all packaging and sizes),Coca-Cola HBC AG,Switzerland,Food & Beverage Processing,1.093,0.1580,38.37%,12.71%,48.93%
2,22710-1-2016,2016,Coca-Cola (all packaging and sizes),Coca-Cola HBC AG,Switzerland,"Food, Beverage & Tobacco",1.093,0.1470,42.17%,10.98%,46.85%
3,3565-10-2013,2013,Coke Zero 330 ml glass bottle,"Coca-Cola Enterprises, Inc.",USA,"Food, Beverage & Tobacco",0.541,0.3400,73.95%,3.42%,22.63%
4,3565-11-2013,2013,Coke Zero 500ml PET,"Coca-Cola Enterprises, Inc.",USA,"Food, Beverage & Tobacco",0.541,0.2200,52.09%,12.32%,35.59%
5,3565-1-2013,2013,Coca-Cola 300ml can,"Coca-Cola Enterprises, Inc.",USA,"Food, Beverage & Tobacco",0.300,0.1700,69.76%,7.00%,23.24%


### A little background:
- Upstream emissions: emissions that occur before the company's own operations such as emissions created by manufacturing bottles that Coke buys from suppliers
- Operations emissions: emissions that the company creates directly, such as when Coke is bottling its product
- Downstream emissions: emissions that occur after the product leaves the company, such as after Coke has sold drinks to McDonald's

## 2: Most recent data

We'll focus on recent emissions data during this code-along. When was the most recent data collected?

- Return the most recent year for which data was collected.

In [26]:
-- Return the most recent year for which data was collected
SELECT MAX(year)
FROM product_emissions;

,max
0,2017


## 3: Targeting major emitters

What are the industries with the most emissions in 2017 (the most recent year that data is available)?

- Return the `industry_group` and a rounded total of `carbon_footprint_pcf` for each industry, aliasing as `total_industry_footprint`.
- Limit to data for 2017 and order by `total_industry_footprint`.

In [27]:
-- Return industry_group and a rounded total of carbon_footprint_pcf, aliased as total_industry_footprint
-- Limit to data for 2017 and order by total_industry_footprint
SELECT industry_group, ROUND(SUM(carbon_footprint_pcf), 1) AS total_industry_footprint
FROM product_emissions
GROUP BY industry_group, year
HAVING year = 2017
ORDER BY total_industry_footprint DESC;

,industry_group,total_industry_footprint
0,Materials,107129.0
1,Capital Goods,94942.7
2,Technology Hardware & Equipment,21865.1
3,"Food, Beverage & Tobacco",3161.5
4,Commercial & Professional Services,740.6
5,Software & Services,690.0


## 4: Industry representation

It looks like the Materials industry had a huge carbon footprint in 2017. But what if that's just because there are many companies from the Materials industry in the dataset? Let's check which industries are most heavily represented in that year.

- Return each `industry_group` included in the table and a count of the number of records that list that industry group.
- Limit the results to only those from 2017 and lias the count as `count_industry`.
- Order by `count_industry`, descending.

In [28]:
-- Return the industry groups and a count of the number of records for each group
-- Limit the results to only those from 2017 and alias the count as count_industry
-- Order by count_industry, descending
SELECT industry_group, COUNT(*) AS count_industry
FROM product_emissions
GROUP BY industry_group, year
HAVING year = 2017
ORDER BY count_industry DESC;

,industry_group,count_industry
0,Technology Hardware & Equipment,22
1,"Food, Beverage & Tobacco",22
2,Materials,11
3,Capital Goods,4
4,Commercial & Professional Services,2
5,Software & Services,1


## 5: Capital Goods industry

We can see that the Materials industry is the biggest emitter _despite_ having less representation in our dataset than several other industries&mdash;yikes! The Capital Goods industry looks similar. Let's explore the companies and products reporting for 2017 in the Capital Goods industry.

- Return `industry_group`, `company`, and `product_name` for all records reporting in the Capital Goods industry during 2017.

In [29]:
-- Return industry_group, company, and product_name for all records reporting in the Capital Goods industry during 2017
SELECT industry_group, company, product_name
FROM product_emissions
WHERE year = 2017
    AND industry_group = 'Capital Goods';

,industry_group,company,product_name
0,Capital Goods,"Mitsui Mining & Smelting Co., Ltd.",Zinc Oxide
1,Capital Goods,"Daikin Industries, Ltd.",Residential Air Conditioner
2,Capital Goods,"Daikin Industries, Ltd.",Commercial Air Conditioner
3,Capital Goods,"Daikin Industries, Ltd.",Light commercial Air Conditioner


## 6: Capital Goods lifecycle emissions

Daikin is an air conditioning and refrigeration manufacturer. Let's look at emissions throughout the life cycle of Daikin products. Can you guess whether most emissions are upstream, downstream, or during operations?

- Return `product_name`, `company`, `upstream_percent_total_pcf`, `operations_percent_total_pcf`, and `downstream_percent_total_pcf` for Daikin Industries, Ltd. in 2017.

In [30]:
-- Return product_name, company, and all stages of pcf emissions for Daikin in 2017
SELECT product_name, 
    company, 
    upstream_percent_total_pcf, 
    operations_percent_total_pcf, 
    downstream_percent_total_pcf
FROM product_emissions
WHERE year = 2017
    AND company = 'Daikin Industries, Ltd.';

,product_name,company,upstream_percent_total_pcf,operations_percent_total_pcf,downstream_percent_total_pcf
0,Residential Air Conditioner,"Daikin Industries, Ltd.",3.96%,0.55%,95.50%
1,Commercial Air Conditioner,"Daikin Industries, Ltd.",0.98%,0.13%,98.88%
2,Light commercial Air Conditioner,"Daikin Industries, Ltd.",0.70%,0.10%,99.21%


## Exploring visually with plotly express

Let's do some quick visualizations with `plotly.express`, a free an open-source graphing library. We don't assume any knowledge of `plotly` on your part!

In [2]:
# Import the plotly express package
import plotly.express as px

## 7: Country representation

Let's take a look at emissions by country. You may have noticed that each time we run a query, the query results are available as `df`. So if we were to select all information from `product_emissions`, that would create a DataFrame called `df` that we can use in a `plotly` visualization! Let's try it.

- Select all information from `product_emissions`.

In [3]:
-- Select all information from product_emissions
SELECT *
FROM product_emissions;

,id,year,product_name,company,country,industry_group,weight_kg,carbon_footprint_pcf,upstream_percent_total_pcf,operations_percent_total_pcf,downstream_percent_total_pcf
0,10056-1-2014,2014,Frosted Flakes(R) Cereal,Kellogg Company,USA,"Food, Beverage & Tobacco",0.748500,2.00,57.50%,30.00%,12.50%
1,10056-1-2015,2015,"Frosted Flakes, 23 oz, produced in Lancaster, ...",Kellogg Company,USA,Food & Beverage Processing,0.748500,2.00,57.50%,30.00%,12.50%
2,10222-1-2013,2013,Office Chair,KNOLL INC,USA,Capital Goods,20.680000,72.54,80.63%,17.36%,2.01%
3,10261-1-2017,2017,Multifunction Printers,"Konica Minolta, Inc.",Japan,Technology Hardware & Equipment,110.000000,1488.00,30.65%,5.51%,63.84%
4,10261-2-2017,2017,Multifunction Printers,"Konica Minolta, Inc.",Japan,Technology Hardware & Equipment,110.000000,1818.00,25.08%,4.51%,70.41%
...,...,...,...,...,...,...,...,...,...,...,...
861,9298-2-2014,2014,Desktop CPU,Intel Corporation,USA,Semiconductors & Semiconductor Equipment,0.408233,14.00,32.14%,54.29%,13.57%
862,9298-3-2013,2013,Mobile CPU,Intel Corporation,USA,Technology Hardware & Equipment,0.408233,7.00,28.57%,71.43%,0.00%
863,9298-3-2014,2014,Mobile CPU,Intel Corporation,USA,Semiconductors & Semiconductor Equipment,0.408233,7.00,31.43%,54.29%,14.29%
864,9792-1-2017,2017,Complete catalyst system for diesel-powered pa...,Johnson Matthey,United Kingdom,Materials,2.000000,188.00,N/a (product with insufficient stage-level data),N/a (product with insufficient stage-level data),N/a (product with insufficient stage-level data)


Let's look at country representation in our dataset by creating a bar chart using the `country` column of `df`.

- Create a plotly barchart of the country distribution for companies in our dataset.

In [9]:
# Create a plotly bar chart of the country distribution for companies in our dataset
px.bar(product_emissions_df, x="country")

## 8: Emissions by country

How does this compare to the emissions of companies reporting from each country?

- First, get the results of interest using SQL: grouping by country, select `country` and the sum of `total carbon_footprint_pcf` by country, aliasing as `total_country_footprint`.

In [7]:
-- Group by country
-- Select country and the sum of total carbon_footprint_pcf by country, aliasing as total_country_footprint
SELECT country, SUM(carbon_footprint_pcf) AS total_country_footprint
FROM product_emissions
GROUP BY country;

,country,total_country_footprint
0,Indonesia,7.210000e+02
1,Switzerland,1.406263e+02
2,Italy,1.940500e+01
3,China,1.425430e+02
4,Luxembourg,1.670073e+05
5,Sweden,4.533010e+03
6,USA,4.518692e+05
7,United Kingdom,6.735140e+03
8,Netherlands,7.041534e+04
9,Brazil,1.675877e+05


- Create a plotly bar plot of the emissions by country in our dataset.

In [8]:
# Create a plotly bar plot of the emissions by country in our dataset
px.bar(pcf_by_country, x="country", y="total_country_footprint")

## 9: Does the graph above make sense?

Wow! Spain has a lot of emissions! Where do they come from? To finish our exploration together, let's take a quick look at the underlying data as a gut-check.

- In SQL, select `company` and `carbon_footprint_pcf` for companies in Spain.

In [38]:
-- Select company and carbon_footprint_pcf for companies in Spain
SELECT company, carbon_footprint_pcf
FROM product_emissions
WHERE country = 'Spain';

,company,carbon_footprint_pcf
0,Miquel Y Costas,3.00
1,"Gamesa Corporación Tecnológica, S.A.",1251625.00
2,"Gamesa Corporación Tecnológica, S.A.",1532608.00
3,"Gamesa Corporación Tecnológica, S.A.",3718044.00
4,"Gamesa Corporación Tecnológica, S.A.",3276187.00
5,"Compañía Española de Petróleos, S.A.U. CEPSA",6109.00
6,"Compañía Española de Petróleos, S.A.U. CEPSA",890.00
7,Crimidesa,180.00
8,Crimidesa,140.00
9,Agraz,155.71


Gamesa Corporación Tecnológica is actually a renewable energy company specializing in wind power! Why might it have such high emissions? Discovering these twists in the data is very important before real analysis can begin. I hope you continue exploring this data on your own!